# 직제순 정렬

# 시각화

In [13]:
pip install pdf2image pillow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install pywin32

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pymupdf

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install pdfkit

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [19]:
pip install pandas matplotlib seaborn plotly

Looking in indexes: https://pypi.org/simple/
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import glob
import os

# 폴더 경로 설정 (일일 데이터와 누계 데이터 경로)
daily_folder_path = r'd:\신규P값총량제\통합'
cumulative_folder_path = r'd:\신규P값총량제\누계통합'

# 파일명에 따른 정렬 기준
file_order = [
    "0. 채널별, 월", "1.기관별, 월",  # "기관별" 파일을 채널 다음으로 정렬
    "2.강북_월", "2.강남_월", "2.부산",
    "2.전남", "2.대구", "2.충남",
    "사원별"
]

# 파일명 정렬 함수
def get_file_priority(file_name):
    for i, keyword in enumerate(file_order):
        if keyword in file_name:
            return i
    return len(file_order)  # 키워드가 없으면 가장 뒤로 정렬

# "제거할 특정 숫자 + 공백 + 건" 패턴을 포함하는 행을 필터링하기 위한 함수 (A열에만 적용)
def filter_out_specific_gun_rows(df):
    remove_patterns = [r'8\s건$', r'12\s건$', r'6\s건$', r'9\s건$', r'7\s건$']
    pattern = '|'.join(remove_patterns)
    return df[~df.iloc[:, 0].astype(str).str.contains(pattern)]

# 폴더에서 파일을 읽어 병합하고 저장하는 함수
def merge_and_save_files(folder_path, output_filename):
    # 모든 파일 경로 가져오기
    all_files = glob.glob(folder_path + "/*.xls")

    # 파일명에 따라 정렬
    all_files.sort(key=lambda x: get_file_priority(os.path.basename(x)))

    # 전체 데이터프레임 초기화
    all_data = pd.DataFrame()

    # 각 파일의 데이터를 순서대로 병합
    for i, file in enumerate(all_files):
        try:
            # HTML 형식으로 파일 읽기
            df_list = pd.read_html(file)
            df = df_list[0]  # 첫 번째 테이블만 사용

            # 특정 "숫자 + 공백 + 건" 패턴을 포함하는 행 제거 (A열만)
            df = filter_out_specific_gun_rows(df)

            # 파일명 출력 및 데이터 미리보기
            if i == 0:
                print(f"{file} 데이터 미리보기:\n", df.head())
            elif i == len(all_files) - 1 or "2.기관별_" in file:
                df = df.iloc[1:]  # 헤더 제외
                print(f"{file} 데이터 미리보기:\n 헤더제외 ->", df.head())
            else:
                print(f"{file} 데이터 미리보기:\n", df.head())

            # 첫 번째 파일에서만 헤더를 사용하고, 이후 파일들은 헤더 없이 추가
            if all_data.empty:
                all_data = df
            else:
                # 기존 열들과 현재 파일의 열을 맞추어 병합
                all_data = pd.concat([all_data, df], ignore_index=True, sort=False)

        except Exception as e:
            print(f"파일을 읽는 중 오류 발생: {file}, 오류: {e}")

    # 병합된 데이터를 csv 파일로 저장 (CP949 인코딩 적용)
    if not all_data.empty:
        csv_output_path = os.path.join(folder_path, output_filename)
        all_data.to_csv(csv_output_path, index=False, encoding='cp949')
        print(f"병합된 파일이 {csv_output_path}에 저장되었습니다.")
    else:
        print("병합된 데이터가 비어 있습니다. 엑셀 파일들을 확인하세요.")

# 일일 데이터 처리
merge_and_save_files(daily_folder_path, "신규P값병합20250112_일일.csv")

# 누계 데이터 처리
merge_and_save_files(cumulative_folder_path, "신규P값병합20250112_누계.csv")


d:\신규P값총량제\통합\0. 채널별, 월.xls 데이터 미리보기:
     0         1          2         3             4
0  채널       월정료   P.P공헌이익금  P.P공헌이익률  P.P(36개월 환산)
1  AM   3855200  102051816     63.63         13.09
2  SC  12532131  175825797     36.62         22.82
3  SP  45810547  523359148     29.44          25.4
4  합계  62197878  801236761     33.13         24.07
d:\신규P값총량제\통합\1.기관별, 월.xls 데이터 미리보기:
          0     1         2          3         4             5
0     실적본부  실적지사       월정료   P.P공헌이익금  P.P공헌이익률  P.P(36개월 환산)
1  강북/강원본부   NaN   9322000  119348360      32.1         24.44
2  강남/서부본부   NaN  10673910  134806295     33.04         24.11
3  부산/경남본부   NaN  14622985  204114911     36.51         22.86
4  대구/경북본부   NaN   5650456   71570290     30.13         25.15
d:\신규P값총량제\통합\2.강북_월.xls 데이터 미리보기:
          0      1        2         3         4             5
0     실적본부   실적지사      월정료  P.P공헌이익금  P.P공헌이익률  P.P(36개월 환산)
1  강북/강원본부   중앙지사   854000  10415780      31.3         24.73
2  강북/강원본부   강북지사   932000

In [3]:
import os
import win32com.client
from datetime import datetime

# 현재 날짜 가져오기 (yyyyMMdd 형식)
current_date = datetime.now().strftime("%Y%m%d")

# PDF 저장 경로 및 파일 이름 지정
output_dir = "D:\\신규P값총량제\\공유메일"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

output_pdf = f"메일용_{current_date}.pdf"
final_pdf = os.path.join(output_dir, output_pdf)

# Excel Application에 연결
excel = win32com.client.Dispatch("Excel.Application")
excel.DisplayAlerts = False  # 모든 경고 비활성화
excel.Visible = False  # Excel 창을 보이지 않게 설정

# PDF 변환 함수 (눈금선 제외)
def export_sheet_to_pdf(sheet, temp_pdf_path, start_row, end_row):
    try:
        # 페이지 설정 - 페이지 영역을 지정하여 각 행 번호 범위에 맞게 설정
        sheet.PageSetup.PrintArea = f"A{start_row}:Z{end_row}"  # A열부터 Z열까지 범위 지정
        sheet.PageSetup.PrintGridlines = False  # 눈금선 제외
        sheet.PageSetup.Zoom = False  # 확대/축소 비활성화
        sheet.PageSetup.FitToPagesWide = 1  # 너비를 한 페이지에 맞춤
        sheet.PageSetup.FitToPagesTall = 1  # 높이를 한 페이지에 맞춤
        sheet.PageSetup.Orientation = 2  # 가로 모드 설정 (xlLandscape = 2)
        sheet.PageSetup.CenterHorizontally = True  # 가로 중앙에 맞춤
        sheet.PageSetup.CenterVertically = True  # 세로 중앙에 맞춤

        # 시트를 PDF로 저장
        sheet.ExportAsFixedFormat(0, temp_pdf_path)
        if os.path.exists(temp_pdf_path) and os.path.getsize(temp_pdf_path) > 0:
            print(f"{sheet.Name} 시트가 성공적으로 PDF로 저장되었습니다: {temp_pdf_path}")
            return True
    except Exception as e:
        print(f"Error converting sheet {sheet.Name} to PDF: {e}")
    return False

try:
    # 열려있는 모든 Workbooks 가져오기
    workbooks = excel.Workbooks
    mail_sheet = None

    # "메일"이라는 이름의 시트를 찾기
    for workbook in workbooks:
        for sheet in workbook.Sheets:
            if sheet.Name == "메일":
                mail_sheet = sheet
                break

    # "메일" 시트가 발견되면 처리
    if mail_sheet:
        # 숨겨진 시트 표시
        mail_sheet.Visible = True
        print(f"시트 {mail_sheet.Name}를 PDF로 변환합니다.")

        # 첫 번째 페이지 (행 1~28) PDF로 변환
        page1_pdf = final_pdf.replace(".pdf", "_page1.pdf")
        if export_sheet_to_pdf(mail_sheet, page1_pdf, start_row=1, end_row=28):
            print(f"첫 번째 페이지가 성공적으로 생성되었습니다: {page1_pdf}")

        # 두 번째 페이지 (행 30~114) PDF로 변환
        page2_pdf = final_pdf.replace(".pdf", "_page2.pdf")
        if export_sheet_to_pdf(mail_sheet, page2_pdf, start_row=30, end_row=114):
            print(f"두 번째 페이지가 성공적으로 생성되었습니다: {page2_pdf}")

            # PDF 파일 열기 (첫 번째 페이지와 두 번째 페이지 모두 열기)
            os.startfile(page1_pdf)
            os.startfile(page2_pdf)
        else:
            print(f"두 번째 페이지 PDF 변환에 실패했습니다.")

    else:
        print("시트 '메일'을 찾을 수 없습니다.")

finally:
    # Excel 종료
    if excel:
        excel.Quit()

    print("작업이 완료되었습니다.")

시트 메일를 PDF로 변환합니다.
메일 시트가 성공적으로 PDF로 저장되었습니다: D:\신규P값총량제\공유메일\메일용_20241031_page1.pdf
첫 번째 페이지가 성공적으로 생성되었습니다: D:\신규P값총량제\공유메일\메일용_20241031_page1.pdf
메일 시트가 성공적으로 PDF로 저장되었습니다: D:\신규P값총량제\공유메일\메일용_20241031_page2.pdf
두 번째 페이지가 성공적으로 생성되었습니다: D:\신규P값총량제\공유메일\메일용_20241031_page2.pdf
작업이 완료되었습니다.


In [5]:
!pip install pdf2image

Defaulting to user installation because normal site-packages is not writeable


In [9]:
!pip install pdf2image

Defaulting to user installation because normal site-packages is not writeable


In [11]:
import os
import win32com.client
from datetime import datetime
from pdf2image import convert_from_path
from PIL import Image

# 현재 날짜 가져오기 (yyyyMMdd 형식)
current_date = datetime.now().strftime("%Y%m%d")

# 이미지 저장 경로 및 파일 이름 지정
output_dir = "D:\\신규P값총량제\\공유메일"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

output_image = f"메일용_{current_date}.jpg"
final_image = os.path.join(output_dir, output_image)

# Excel Application에 연결
excel = win32com.client.Dispatch("Excel.Application")
excel.DisplayAlerts = False  # 모든 경고 비활성화
excel.Visible = False  # Excel 창을 보이지 않게 설정

# 이미지 변환 함수 (눈금선 제외)
def export_sheet_to_image(sheet, temp_image_path, start_row, end_row):
    try:
        # 페이지 설정 - 페이지 영역을 지정하여 각 행 번호 범위에 맞게 설정
        sheet.PageSetup.PrintArea = f"A{start_row}:Z{end_row}"  # A열부터 Z열까지 범위 지정
        sheet.PageSetup.PrintGridlines = False  # 눈금선 제외
        sheet.PageSetup.Zoom = False  # 확대/축소 비활성화
        sheet.PageSetup.FitToPagesWide = 1  # 너비를 한 페이지에 맞춤
        sheet.PageSetup.FitToPagesTall = 1  # 높이를 한 페이지에 맞춤
        sheet.PageSetup.Orientation = 2  # 가로 모드 설정 (xlLandscape = 2)
        sheet.PageSetup.CenterHorizontally = True  # 가로 중앙에 맞춤
        sheet.PageSetup.CenterVertically = True  # 세로 중앙에 맞춤

        # 시트를 PDF로 임시 저장 후 이미지로 변환
        temp_pdf_path = temp_image_path.replace(".jpg", ".pdf")
        sheet.ExportAsFixedFormat(0, temp_pdf_path)

        # PDF를 이미지로 변환
        images = convert_from_path(temp_pdf_path)
        images[0].save(temp_image_path, "JPEG")
        os.remove(temp_pdf_path)  # 임시 PDF 파일 삭제

        if os.path.exists(temp_image_path) and os.path.getsize(temp_image_path) > 0:
            print(f"{sheet.Name} 시트가 성공적으로 이미지로 저장되었습니다: {temp_image_path}")
            return True
    except Exception as e:
        print(f"Error converting sheet {sheet.Name} to image: {e}")
    return False

try:
    # 열려있는 모든 Workbooks 가져오기
    workbooks = excel.Workbooks
    mail_sheet = None

    # "메일"이라는 이름의 시트를 찾기
    for workbook in workbooks:
        for sheet in workbook.Sheets:
            if sheet.Name == "메일":
                mail_sheet = sheet
                break

    # "메일" 시트가 발견되면 처리
    if mail_sheet:
        # 숨겨진 시트 표시
        mail_sheet.Visible = True
        print(f"시트 {mail_sheet.Name}를 이미지로 변환합니다.")

        # 첫 번째 페이지 (행 1~28) 이미지를 저장
        page1_image = final_image.replace(".jpg", "_page1.jpg")
        if export_sheet_to_image(mail_sheet, page1_image, start_row=1, end_row=28):
            print(f"첫 번째 페이지가 성공적으로 생성되었습니다: {page1_image}")

        # 두 번째 페이지 (행 30~114) 이미지를 저장
        page2_image = final_image.replace(".jpg", "_page2.jpg")
        if export_sheet_to_image(mail_sheet, page2_image, start_row=30, end_row=114):
            print(f"두 번째 페이지가 성공적으로 생성되었습니다: {page2_image}")

            # 이미지 파일 열기
            os.startfile(page1_image)
            os.startfile(page2_image)
        else:
            print(f"두 번째 페이지 이미지 변환에 실패했습니다.")

    else:
        print("시트 '메일'을 찾을 수 없습니다.")

finally:
    # Excel 종료
    if excel:
        excel.Quit()

    print("작업이 완료되었습니다.")

ModuleNotFoundError: No module named 'pdf2image'

# 한번에 실행 다시 (10.10)  작업 엑셀파일을 열어놓고 실행

In [2]:
import os
import pandas as pd
import win32com.client
from datetime import datetime

# 폴더 경로 설정
daily_folder_path = r'd:\신규P값총량제\병합'
cumulative_folder_path = r'd:\신규P값총량제\누계병합'

# 파일명에 따른 정렬 기준 설정
file_order = [
    "0. 채널별, 월", "1.기관별, 월",
    "2.강북_월", "2.강남_월", "2.부산",
    "2.전남", "2.대구", "2.충남",
    "사원별"
]

# 파일명 정렬 함수
def get_file_priority(file_name):
    for i, keyword in enumerate(file_order):
        if keyword in file_name:
            return i
    return len(file_order)

# 특정 행을 필터링하여 제거 (A열에만 적용)
def filter_out_specific_gun_rows(df):
    remove_patterns = [r'8\s건$', r'12\s건$', r'6\s건$', r'9\s건$', r'7\s건$']
    pattern = '|'.join(remove_patterns)
    return df[~df.iloc[:, 0].astype(str).str.contains(pattern)]

# CSV 파일 병합 함수
def merge_and_save_files(folder_path, output_filename):
    all_files = sorted(glob.glob(folder_path + "/*.xls"), key=lambda x: get_file_priority(os.path.basename(x)))
    all_data = pd.DataFrame()
    for file in all_files:
        try:
            df_list = pd.read_html(file)
            df = df_list[0]
            df = filter_out_specific_gun_rows(df)

            if all_data.empty:
                all_data = df
            else:
                all_data = pd.concat([all_data, df], ignore_index=True, sort=False)
        except Exception as e:
            print(f"파일 읽기 오류: {file}, 오류: {e}")

    if not all_data.empty:
        csv_output_path = os.path.join(folder_path, output_filename)
        all_data.to_csv(csv_output_path, index=False, encoding='cp949')
        print(f"병합된 파일이 {csv_output_path}에 저장되었습니다.")
        return csv_output_path
    else:
        print("병합된 데이터가 비어 있습니다.")
        return None

# 엑셀 데이터 백그라운드 복사 함수
def paste_data_to_excel(sheet, start_cell, data):
    for i, row in enumerate(data, start=start_cell[0]):
        for j, value in enumerate(row, start=start_cell[1]):
            if not sheet.Cells(i, j).HasFormula:
                sheet.Cells(i, j).Value = value

# Excel 데이터 처리
def copy_data_to_excel():
    try:
        excel = win32com.client.Dispatch("Excel.Application")
        excel.Visible = False
        workbook = excel.Workbooks(1)

        # 엑셀 파일 열려 있는 상태 확인
        if not workbook:
            raise Exception("엑셀 파일을 열어주세요.")

        pv_db_sheet = workbook.Sheets("Pv_db")
        cumulative_sheet = workbook.Sheets("92_SP사원(개인)(10월 누계)")
        channel_sheet = workbook.Sheets("91_기관&채널(AM-SC-SP)(10월누계)")

        # CSV 파일 병합 후 저장
        daily_csv = merge_and_save_files(daily_folder_path, "신규P값병합1010_일일.csv")
        cumulative_csv = merge_and_save_files(cumulative_folder_path, "신규P값병합08월_10월_1010_누계.csv")

        # CSV 파일 열기 및 데이터 가져오기
        daily_data = pd.read_csv(daily_csv, encoding='cp949').values
        cumulative_data = pd.read_csv(cumulative_csv, encoding='cp949').values

        # 데이터 백그라운드 복사
        paste_data_to_excel(pv_db_sheet, (2, 2), cumulative_data)  # B열부터
        paste_data_to_excel(pv_db_sheet, (2, 13), daily_data)  # M열부터

        # 특정 조건에 맞는 값 복사
        for row in range(2, pv_db_sheet.UsedRange.Rows.Count + 1):
            if "본사" in str(pv_db_sheet.Cells(row, 13).Value):
                for col in range(13, 21):
                    cumulative_sheet.Cells(2, col - 11).Value = pv_db_sheet.Cells(row, col).Value

        # 엑셀 파일을 저장하고 상태 확인
        current_date = datetime.now().strftime("%Y%m%d")
        new_filename = f"총량제 실적현황_{current_date}.xlsx"
        save_path = os.path.join(os.path.dirname(workbook.FullName), new_filename)
        workbook.SaveAs(save_path)

        return excel, workbook, save_path
    except Exception as e:
        print(f"오류 발생: {e}")
        return None, None, None

# PDF 변환 및 저장
def export_sheet_to_pdf(excel, workbook):
    try:
        output_pdf = os.path.join("D:\\신규P값총량제\\공유메일", f"메일용_{datetime.now().strftime('%Y%m%d')}.pdf")
        mail_sheet = workbook.Sheets("메일")
        mail_sheet.Visible = True
        mail_sheet.ExportAsFixedFormat(0, output_pdf)

        print(f"PDF 파일이 성공적으로 생성되었습니다: {output_pdf}")
        os.startfile(output_pdf)
    except Exception as e:
        print(f"PDF 변환 오류: {e}")

# 메인 실행 함수
if __name__ == "__main__":
    excel_app, open_workbook, excel_file_path = copy_data_to_excel()
    if excel_app and open_workbook:
        export_sheet_to_pdf(excel_app, open_workbook)

오류 발생: (-2147352567, '예외가 발생했습니다.', (0, None, None, None, 0, -2147352565), None)


In [6]:
import win32com.client

# 엑셀 애플리케이션 객체 생성

excel = win32com.client.Dispatch("Excel.Application")

excel.Visible = True



# 엑셀 파일 열기

workbook_path = r'd:\신규P값총량제\연습2.xlsx'

try:

    workbook = excel.Workbooks.Open(workbook_path)

    print(f"엑셀 파일 '{workbook_path}'이 성공적으로 열렸습니다.")

except Exception as e:

    print(f"엑셀 파일을 열 수 없습니다: {e}")

엑셀 파일 'd:\신규P값총량제\연습2.xlsx'이 성공적으로 열렸습니다.


In [5]:
if not excel:

    raise Exception("Excel 애플리케이션이 제대로 생성되지 않았습니다.")

In [7]:
import os



# 모든 Excel 프로세스 종료

os.system("taskkill /f /im excel.exe")

print("모든 Excel 프로세스가 종료되었습니다.")

모든 Excel 프로세스가 종료되었습니다.


## 진짜성공

In [11]:
import os
import pandas as pd
import glob
import win32com.client
from datetime import datetime
import time

# 파일 경로 설정
excel_file_path = r'd:\신규P값총량제\연습2.xlsx'  # 자동으로 열릴 파일 경로
daily_folder_path = r'd:\신규P값총량제\병합'
cumulative_folder_path = r'd:\신규P값총량제\누계병합'

# 파일명 정렬 기준
file_order = [
    "0. 채널별, 월", "1.기관별, 월",
    "2.강북_월", "2.강남_월", "2.부산",
    "2.전남", "2.대구", "2.충남",
    "사원별"
]

# 파일 정렬 함수
def get_file_priority(file_name):
    for i, keyword in enumerate(file_order):
        if keyword in file_name:
            return i
    return len(file_order)  # 키워드가 없으면 가장 뒤로 정렬

# CSV 병합 및 저장
def merge_and_save_files(folder_path, output_filename):
    all_files = glob.glob(folder_path + "/*.xls")
    all_files.sort(key=lambda x: get_file_priority(os.path.basename(x)))

    all_data = pd.DataFrame()

    for file in all_files:
        try:
            df_list = pd.read_html(file)
            df = df_list[0]

            if all_data.empty:
                all_data = df
            else:
                all_data = pd.concat([all_data, df], ignore_index=True, sort=False)

        except Exception as e:
            print(f"파일을 읽는 중 오류 발생: {file}, 오류: {e}")

    if not all_data.empty:
        csv_output_path = os.path.join(folder_path, output_filename)
        all_data.to_csv(csv_output_path, index=False, encoding='cp949')
        print(f"병합된 파일이 {csv_output_path}에 저장되었습니다.")
        return csv_output_path
    else:
        print("병합된 데이터가 비어 있습니다.")
        return None

# CSV 데이터 엑셀에 붙여넣기
def paste_dataframe_to_excel(sheet, start_cell, df):
    values = df.fillna("").values.tolist()
    start_row, start_col = start_cell
    end_row = start_row + len(values) - 1
    end_col = start_col + len(values[0]) - 1

    excel_range = sheet.Range(sheet.Cells(start_row, start_col), sheet.Cells(end_row, end_col))
    excel_range.Value = values

# Excel 데이터 병합 및 붙여넣기
def copy_data_to_excel():
    try:
        # Excel 파일 열기
        excel = win32com.client.Dispatch("Excel.Application")
        excel.Visible = True
        workbook = excel.Workbooks.Open(excel_file_path)

        # 엑셀에서 필요한 시트 가져오기
        pv_db_sheet = workbook.Sheets("Pv_db")
        cumulative_sheet = workbook.Sheets("92_SP사원(개인)(10월 누계)")
        channel_sheet = workbook.Sheets("91_기관&채널(AM-SC-SP)(10월누계)")

        # 병합된 파일 생성 및 저장
        daily_csv = merge_and_save_files(daily_folder_path, "신규P값병합_일일.csv")
        cumulative_csv = merge_and_save_files(cumulative_folder_path, "신규P값병합_누계.csv")

        if not daily_csv or not cumulative_csv:
            raise Exception("CSV 병합 파일 생성 중 오류가 발생했습니다.")

        daily_data = pd.read_csv(daily_csv, encoding='cp949')
        cumulative_data = pd.read_csv(cumulative_csv, encoding='cp949')

        # 누계 데이터 Pv_db 시트에 복사
        paste_dataframe_to_excel(pv_db_sheet, (1, 2), cumulative_data)

        # 일일 데이터 Pv_db 시트에 복사
        paste_dataframe_to_excel(pv_db_sheet, (1, 13), daily_data)

        # 특정 조건에 맞는 행을 누계 시트에 복사
        for row in range(2, pv_db_sheet.UsedRange.Rows.Count + 1):
            if "본사" in str(pv_db_sheet.Cells(row, 13).Value):
                for col in range(13, 21):
                    cumulative_sheet.Cells(2, col - 11).Value = pv_db_sheet.Cells(row, col).Value
                break

        # Pv_db 시트 값 복사해서 채널 시트에 적용
        for row in range(2, 6):
            for col_offset in range(4):
                channel_sheet.Cells(row + 1, 18 + col_offset).Value = pv_db_sheet.Cells(row, 14 + col_offset).Value

        # 엑셀 파일 저장
        current_date = datetime.now().strftime("%Y%m%d")
        new_filename = f"총량제 실적현황_{current_date}.xlsx"
        save_path = os.path.join(os.path.dirname(workbook.FullName), new_filename)
        workbook.SaveAs(save_path)

        print(f"데이터 병합 및 엑셀 저장 완료. 파일명: {save_path}")
        return excel, workbook, save_path
    except Exception as e:
        print(f"작업 중 오류 발생: {e}")
        return None, None, None

# PDF 변환 함수
def export_sheet_to_pdf(excel, workbook, sheet_name, output_path):
    try:
        sheet = workbook.Sheets(sheet_name)
        sheet.PageSetup.Zoom = False
        sheet.PageSetup.FitToPagesWide = 1
        sheet.PageSetup.FitToPagesTall = 1
        sheet.ExportAsFixedFormat(0, output_path)
        print(f"PDF 파일이 성공적으로 생성되었습니다: {output_path}")
    except Exception as e:
        print(f"PDF 변환 중 오류 발생: {e}")

# 전체 프로세스 실행
if __name__ == "__main__":
    excel_app, open_workbook, excel_file_path = copy_data_to_excel()
    if excel_app and open_workbook and excel_file_path:
        pdf_output = os.path.join("D:\\신규P값총량제\\공유메일", f"메일용_{datetime.now().strftime('%Y%m%d')}.pdf")
        export_sheet_to_pdf(excel_app, open_workbook, "메일", pdf_output)

병합된 파일이 d:\신규P값총량제\병합\신규P값병합_일일.csv에 저장되었습니다.
병합된 파일이 d:\신규P값총량제\누계병합\신규P값병합_누계.csv에 저장되었습니다.
데이터 병합 및 엑셀 저장 완료. 파일명: d:\신규P값총량제\총량제 실적현황_20241011.xlsx
PDF 파일이 성공적으로 생성되었습니다: D:\신규P값총량제\공유메일\메일용_20241011.pdf


In [ ]:
10.11 

In [16]:
import os
import pandas as pd
import glob
import win32com.client
from datetime import datetime
import time

# 파일 경로 설정
excel_file_path = r'd:\신규P값총량제\연습4.xlsx'  # 자동으로 열릴 파일 경로
daily_folder_path = r'd:\신규P값총량제\병합'
cumulative_folder_path = r'd:\신규P값총량제\누계병합'

# 파일명 정렬 기준
file_order = [
    "0. 채널별, 월", "1.기관별, 월",
    "2.강북_월", "2.강남_월", "2.부산",
    "2.전남", "2.대구", "2.충남",
    "사원별"
]

# 파일 정렬 함수
def get_file_priority(file_name):
    for i, keyword in enumerate(file_order):
        if keyword in file_name:
            return i
    return len(file_order)  # 키워드가 없으면 가장 뒤로 정렬

# CSV 병합 및 저장
def merge_and_save_files(folder_path, output_filename):
    all_files = glob.glob(folder_path + "/*.xls")
    all_files.sort(key=lambda x: get_file_priority(os.path.basename(x)))

    all_data = pd.DataFrame()

    for file in all_files:
        try:
            df_list = pd.read_html(file)
            df = df_list[0]

            if all_data.empty:
                all_data = df
            else:
                all_data = pd.concat([all_data, df], ignore_index=True, sort=False)

        except Exception as e:
            print(f"파일을 읽는 중 오류 발생: {file}, 오류: {e}")

    if not all_data.empty:
        csv_output_path = os.path.join(folder_path, output_filename)
        all_data.to_csv(csv_output_path, index=False, encoding='cp949')
        print(f"병합된 파일이 {csv_output_path}에 저장되었습니다.")
        return csv_output_path
    else:
        print("병합된 데이터가 비어 있습니다.")
        return None

# CSV 데이터 엑셀에 붙여넣기
def paste_dataframe_to_excel(sheet, start_cell, df):
    values = df.fillna("").values.tolist()
    start_row, start_col = start_cell
    for i, row in enumerate(values):
        for j, value in enumerate(row):
            sheet.Cells(start_row + i, start_col + j).Value = value

# Excel 데이터 병합 및 붙여넣기
def copy_data_to_excel():
    try:
        # Excel 파일 열기
        excel = win32com.client.Dispatch("Excel.Application")
        excel.Visible = True
        workbook = excel.Workbooks.Open(excel_file_path)

        # 엑셀에서 필요한 시트 가져오기
        pv_db_sheet = workbook.Sheets("Pv_db")
        cumulative_sheet = workbook.Sheets("92_SP사원(개인)(10월 누계)")
        channel_sheet = workbook.Sheets("91_기관&채널(AM-SC-SP)(10월누계)")

        # 병합된 파일 생성 및 저장
        daily_csv = merge_and_save_files(daily_folder_path, "신규P값병합_일일.csv")
        cumulative_csv = merge_and_save_files(cumulative_folder_path, "신규P값병합_누계.csv")

        if not daily_csv or not cumulative_csv:
            raise Exception("CSV 병합 파일 생성 중 오류가 발생했습니다.")

        daily_data = pd.read_csv(daily_csv, encoding='cp949')
        cumulative_data = pd.read_csv(cumulative_csv, encoding='cp949')

        # 누계 데이터 Pv_db 시트에 복사
        paste_dataframe_to_excel(pv_db_sheet, (1, 2), cumulative_data)

        # 일일 데이터 Pv_db 시트에 복사
        paste_dataframe_to_excel(pv_db_sheet, (1, 13), daily_data)

        # 특정 조건에 맞는 행을 누계 시트에 복사
        for row in range(2, pv_db_sheet.UsedRange.Rows.Count + 1):
            if "본사" in str(pv_db_sheet.Cells(row, 13).Value):
                for col in range(13, 21):
                    cumulative_sheet.Cells(2, col - 11).Value = pv_db_sheet.Cells(row, col).Value
                break

        # Pv_db 시트 값 복사해서 채널 시트에 적용
        for row in range(2, 6):
            for col_offset in range(4):
                channel_sheet.Cells(row + 1, 18 + col_offset).Value = pv_db_sheet.Cells(row, 14 + col_offset).Value

        # 엑셀 파일 저장
        current_date = datetime.now().strftime("%Y%m%d")
        new_filename = f"총량제 실적현황_{current_date}.xlsx"
        save_path = os.path.join(os.path.dirname(workbook.FullName), new_filename)
        workbook.SaveAs(save_path)

        print(f"데이터 병합 및 엑셀 저장 완료. 파일명: {save_path}")
        return excel, workbook, save_path
    except Exception as e:
        print(f"작업 중 오류 발생: {e}")
        return None, None, None

# PDF 변환 함수 및 파일 열기 (중앙에 위치)
def export_sheet_to_pdf(excel, workbook, sheet_name, output_path):
    try:
        sheet = workbook.Sheets(sheet_name)
        sheet.PageSetup.Zoom = False
        sheet.PageSetup.FitToPagesWide = 1
        sheet.PageSetup.FitToPagesTall = 1
        sheet.PageSetup.CenterHorizontally = True
        sheet.PageSetup.CenterVertically = True
        sheet.ExportAsFixedFormat(0, output_path)
       
        # PDF 파일을 열 때 중앙에 오도록 설정
        os.system(f'start "" "{output_path}"')
        print(f"PDF 파일이 성공적으로 생성되었습니다: {output_path}")
    except Exception as e:
        print(f"PDF 변환 중 오류 발생: {e}")

# 전체 프로세스 실행
if __name__ == "__main__":
    excel_app, open_workbook, excel_file_path = copy_data_to_excel()
    if excel_app and open_workbook and excel_file_path:
        pdf_output = os.path.join("D:\\신규P값총량제\\공유메일", f"메일용_{datetime.now().strftime('%Y%m%d')}.pdf")
        export_sheet_to_pdf(excel_app, open_workbook, "메일", pdf_output)

병합된 파일이 d:\신규P값총량제\병합\신규P값병합_일일.csv에 저장되었습니다.
병합된 파일이 d:\신규P값총량제\누계병합\신규P값병합_누계.csv에 저장되었습니다.
작업 중 오류 발생: (-2147352567, '예외가 발생했습니다.', (0, None, None, None, 0, -2146777998), None)


In [17]:
import os
import pandas as pd
import openpyxl
import win32com.client as win32

# 파일 경로 설정
base_folder = r'd:\신규P값총량제'
original_file = os.path.join(base_folder, "연습4.xl")  # 확장자가 .xl로 변경된 파일 경로
new_file_path = os.path.join(base_folder, "연습4.xlsx")  # 변경할 새로운 확장자

# 파일 확장자 변경
if os.path.exists(original_file):
    os.rename(original_file, new_file_path)
    print(f"파일 확장자가 .xl에서 .xlsx로 변경되었습니다: {new_file_path}")
else:
    print(f"지정된 파일이 존재하지 않습니다: {original_file}")

# 확장자가 변경된 파일을 Pandas로 읽기
try:
    df = pd.read_excel(new_file_path, sheet_name=None)  # 모든 시트 읽기
    print(f"엑셀 파일이 성공적으로 읽어졌습니다. 시트 목록: {list(df.keys())}")
except Exception as e:
    print(f"파일을 읽는 중 오류 발생: {e}")

# 엑셀 애플리케이션 열기 및 작업 확인
try:
    # 엑셀 애플리케이션 열기
    excel = win32.Dispatch("Excel.Application")
    excel.Visible = True
    excel.DisplayAlerts = False

    # 엑셀 파일 열기
    workbook = excel.Workbooks.Open(new_file_path)

    # 시트 목록 출력
    sheet_names = [sheet.Name for sheet in workbook.Sheets]
    print(f"현재 엑셀 파일의 시트 목록: {sheet_names}")

    # 파일 작업 예시 (첫 번째 시트의 셀 값 확인)
    sheet = workbook.Sheets(1)
    first_cell_value = sheet.Cells(1, 1).Value
    print(f"첫 번째 시트의 첫 번째 셀 값: {first_cell_value}")

    # 엑셀 파일을 저장하고 종료
    workbook.Save()
    workbook.Close()
    excel.Quit()
    print("엑셀 파일 작업이 완료되었습니다.")

except Exception as e:
    print(f"엑셀 파일 작업 중 오류 발생: {e}")


지정된 파일이 존재하지 않습니다: d:\신규P값총량제\연습4.xl
엑셀 파일이 성공적으로 읽어졌습니다. 시트 목록: ['메일', '10_(기관) 누계요약', '11_(기관)신규P값 성과', '12_(SP개인)신규P값 성과', '본사실적_고지점령', '시각화', '51_월간 목표 비 실적(집계표)', '| Raw Data ▶▶▶ |', 'Pv_db', '91_기관&채널(AM-SC-SP)(10월누계)', '92_SP사원(개인)(10월 누계)', '98_P.P GL', '99_서비스 분류체계', '8월 적자구역 확정내역']
현재 엑셀 파일의 시트 목록: ['메일', '10_(기관) 누계요약', '11_(기관)신규P값 성과', '12_(SP개인)신규P값 성과', '본사실적_고지점령', '시각화', '51_월간 목표 비 실적(집계표)', '| Raw Data ▶▶▶ |', 'Pv_db', '91_기관&채널(AM-SC-SP)(10월누계)', '92_SP사원(개인)(10월 누계)', '98_P.P GL', '99_서비스 분류체계', '8월 적자구역 확정내역']
첫 번째 시트의 첫 번째 셀 값: None
엑셀 파일 작업이 완료되었습니다.


In [18]:
import os

# 원본 파일 경로와 변경할 파일 경로 설정
original_file = r'd:\신규P값총량제\연습4.xlsx'
new_file = r'd:\신규P값총량제\연습.xl'

# 파일 이름 변경
try:
    if os.path.exists(original_file):
        os.rename(original_file, new_file)
        print(f"파일 이름이 성공적으로 변경되었습니다: {original_file} -> {new_file}")
    else:
        print(f"지정된 파일이 존재하지 않습니다: {original_file}")
except Exception as e:
    print(f"파일 이름 변경 중 오류 발생: {e}")


파일 이름이 성공적으로 변경되었습니다: d:\신규P값총량제\연습4.xlsx -> d:\신규P값총량제\연습.xl


In [19]:
import os
import win32com.client

# 원본 파일 경로와 변경할 파일 경로 설정
original_file = r'd:\신규P값총량제\연습.xl'
new_file = r'd:\신규P값총량제\연습.xlsx'

# 파일 이름 변경 (확장자 변경)
try:
    if os.path.exists(original_file):
        os.rename(original_file, new_file)
        print(f"파일 이름이 성공적으로 변경되었습니다: {original_file} -> {new_file}")
    else:
        print(f"지정된 파일이 존재하지 않습니다: {original_file}")
except Exception as e:
    print(f"파일 이름 변경 중 오류 발생: {e}")

# Excel 파일 열기 및 시트 작업
try:
    excel = win32com.client.Dispatch("Excel.Application")
    excel.Visible = True
    workbook = excel.Workbooks.Open(new_file)
   
    # 모든 시트 이름 확인
    sheet_names = [sheet.Name for sheet in workbook.Sheets]
    print(f"엑셀 파일의 시트 목록: {sheet_names}")

    # 특정 시트 선택 및 작업
    if "Pv_db" in sheet_names:
        sheet = workbook.Sheets("Pv_db")
        print(f"'Pv_db' 시트의 A1 셀 값: {sheet.Cells(1, 1).Value}")
    else:
        print("'Pv_db' 시트가 존재하지 않습니다.")
   
    # Excel 저장 후 종료
    workbook.Save()
    workbook.Close()
    excel.Quit()
except Exception as e:
    print(f"엑셀 파일 작업 중 오류 발생: {e}")

파일 이름이 성공적으로 변경되었습니다: d:\신규P값총량제\연습.xl -> d:\신규P값총량제\연습.xlsx
엑셀 파일의 시트 목록: ['메일', '10_(기관) 누계요약', '11_(기관)신규P값 성과', '12_(SP개인)신규P값 성과', '본사실적_고지점령', '시각화', '51_월간 목표 비 실적(집계표)', '| Raw Data ▶▶▶ |', 'Pv_db', '91_기관&채널(AM-SC-SP)(10월누계)', '92_SP사원(개인)(10월 누계)', '98_P.P GL', '99_서비스 분류체계', '8월 적자구역 확정내역']
'Pv_db' 시트의 A1 셀 값: 누계


## 위 코드 실행 후 아래 코드 실행(pdf 자동생성 및 열기)_10.10성공

In [2]:
import os
import win32com.client
from datetime import datetime

# 현재 날짜 가져오기 (yyyyMMdd 형식)
current_date = datetime.now().strftime("%Y%m%d")

# PDF 저장 경로 및 파일 이름 지정
output_dir = "D:\\신규P값총량제\\공유메일"
output_pdf = f"메일용_{current_date}.pdf"  # 현재 날짜를 포함한 파일 이름
final_pdf = os.path.join(output_dir, output_pdf)

# Excel Application에 연결
excel = win32com.client.Dispatch("Excel.Application")
excel.DisplayAlerts = False  # 모든 경고 비활성화
excel.Visible = False  # Excel 창을 보이지 않게 설정

# PDF 변환 함수 (눈금선 제외)
def export_sheet_to_pdf(sheet, temp_pdf_path):
    try:
        # 현재 보이는 열과 셀만 PDF로 변환하도록 설정, 눈금선 제외
        sheet.PageSetup.PrintArea = ""  # 인쇄 영역 설정을 초기화
        sheet.PageSetup.PrintGridlines = False  # 눈금선 제외

        # 시트를 PDF로 저장
        sheet.ExportAsFixedFormat(0, temp_pdf_path)
        if os.path.exists(temp_pdf_path) and os.path.getsize(temp_pdf_path) > 0:
            print(f"{sheet.Name} 시트가 성공적으로 PDF로 저장되었습니다: {temp_pdf_path}")
            return True
    except Exception as e:
        print(f"Error converting sheet {sheet.Name} to PDF: {e}")
    return False

try:
    # 열려있는 모든 Workbooks 가져오기
    workbooks = excel.Workbooks
    mail_sheet = None

    # "메일"이라는 이름의 시트를 찾기
    for workbook in workbooks:
        for sheet in workbook.Sheets:
            if sheet.Name == "메일":
                mail_sheet = sheet
                break

    # "메일" 시트가 발견되면 처리
    if mail_sheet:
        # 숨겨진 시트 표시
        mail_sheet.Visible = True
        print(f"시트 {mail_sheet.Name}를 PDF로 변환합니다.")

        # PDF 변환
        if export_sheet_to_pdf(mail_sheet, final_pdf):
            print(f"PDF 파일이 성공적으로 생성되었습니다: {final_pdf}")

            # PDF 파일 열기
            os.startfile(final_pdf)
        # 변환 후 시트를 다시 숨기기
        mail_sheet.Visible = False
    else:
        print("시트 '메일'을 찾을 수 없습니다.")

finally:
    # Excel 종료
    if excel:
        excel.Quit()

    print("작업이 완료되었습니다.")

시트 메일를 PDF로 변환합니다.
메일 시트가 성공적으로 PDF로 저장되었습니다: D:\신규P값총량제\공유메일\메일용_20241010.pdf
PDF 파일이 성공적으로 생성되었습니다: D:\신규P값총량제\공유메일\메일용_20241010.pdf
작업이 완료되었습니다.


In [14]:
import win32com.client

# 기존 실행 중인 엑셀 인스턴스 찾기
try:
    excel = win32com.client.GetObject(Class="Excel.Application")
    print("기존 엑셀 인스턴스를 찾았습니다.")
except:
    excel = win32com.client.Dispatch("Excel.Application")
    print("새 엑셀 인스턴스를 생성했습니다.")

excel.Visible = True

기존 엑셀 인스턴스를 찾았습니다.


In [15]:
excel.Workbooks.Open(r'd:\신규P값총량제\연습3.xlsx')

<COMObject Open>

In [13]:
def reset_and_optimize_excel():
    # 모든 엑셀 프로세스를 종료
    reset_excel()
   
    # 새로운 엑셀 인스턴스 생성 및 보안 설정 최적화
    excel = win32com.client.Dispatch("Excel.Application")
    configure_excel_security_settings(excel)
   
    # 엑셀 파일 열기 및 속성 최적화
    try:
        excel.Visible = True
        excel.DisplayAlerts = False
        excel.ScreenUpdating = True
        print("엑셀이 정상적으로 복구되었습니다.")
    except Exception as e:
        print(f"엑셀 복구 중 오류 발생: {e}")

In [16]:
import win32com.client
import os
import time

def reset_excel_process():
    try:
        # 모든 실행 중인 엑셀 프로세스를 종료
        os.system('taskkill /f /im excel.exe')
        print("모든 엑셀 프로세스가 종료되었습니다.")
        time.sleep(3)  # 모든 프로세스가 완전히 종료될 때까지 잠시 대기
    except Exception as e:
        print(f"엑셀 프로세스 종료 중 오류 발생: {e}")

def create_stable_excel_instance():
    try:
        # 새로운 엑셀 인스턴스 생성
        excel = win32com.client.DispatchEx("Excel.Application")  # DispatchEx로 새로운 인스턴스 생성
        excel.Visible = True
        excel.DisplayAlerts = False
        print("새로운 엑셀 인스턴스가 정상적으로 생성되었습니다.")
        return excel
    except Exception as e:
        print(f"엑셀 인스턴스 생성 중 오류 발생: {e}")
        return None

def main():
    # 엑셀 프로세스 초기화
    reset_excel_process()
   
    # 안정적인 엑셀 인스턴스 생성
    excel = create_stable_excel_instance()
   
    if not excel:
        print("엑셀 초기화 실패. 작업을 중단합니다.")
        return

    # 여기서 추가 작업 수행 가능

    # 엑셀 인스턴스 종료
    excel.Quit()
    print("엑셀 작업이 정상적으로 종료되었습니다.")

if __name__ == "__main__":
    main()

모든 엑셀 프로세스가 종료되었습니다.
새로운 엑셀 인스턴스가 정상적으로 생성되었습니다.
엑셀 작업이 정상적으로 종료되었습니다.


In [17]:
pip install --upgrade pywin32

Looking in indexes: https://pypi.org/simple/
   ---------------------------------------- 0.0/6.5 MB ? eta -:--:--
   ------ --------------------------------- 1.0/6.5 MB 20.7 MB/s eta 0:00:01
   ------------- -------------------------- 2.2/6.5 MB 23.2 MB/s eta 0:00:01
   ------------------- -------------------- 3.2/6.5 MB 23.0 MB/s eta 0:00:01
   --------------------------- ------------ 4.4/6.5 MB 23.7 MB/s eta 0:00:01
   ---------------------------------------  6.5/6.5 MB 23.1 MB/s eta 0:00:01
   ---------------------------------------  6.5/6.5 MB 23.1 MB/s eta 0:00:01
   ---------------------------------------- 6.5/6.5 MB 17.4 MB/s eta 0:00:00
  Attempting uninstall: pywin32
    Found existing installation: pywin32 306
    Uninstalling pywin32-306:
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'c:\\users\\user\\appdata\\roaming\\python\\python312\\site-packages\\win32\\win32api.pyd'
Consider using the `--user` option or check the permissions.

